# Tool Usage Examples

This notebook demonstrates how to use the OpenAPI to Pydantic-AI tools converter with practical examples from both the AI Knowledge API and Ally Config API.

In [1]:
# Setup Python path to import from src directory
import sys
import os
from pathlib import Path

# Add the parent directory (project root) to Python path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print(f"✅ Added {project_root} to Python path")
print(f"Current working directory: {Path.cwd()}")
print(f"Python path includes: {project_root}")

✅ Added /home/jschillb/meta_ally to Python path
Current working directory: /home/jschillb/meta_ally/examples
Python path includes: /home/jschillb/meta_ally


In [2]:
# Import required libraries
import asyncio
from src.lib.openapi_to_tools import OpenAPIToolsLoader
from pydantic_ai import Agent, RunContext
from typing import Any

# We'll use these for our examples
print("✅ Imports loaded successfully!")

✅ Imports loaded successfully!


In [3]:
# Load tools from Ally Config API
from src.lib.auth_manager import AuthManager


print("Loading Ally Config API tools...")
auth_manager = AuthManager(
    keycloak_url="https://keycloak.acc.iam-services.aws.inform-cloud.io/",
    realm_name="inform-ai",
    client_id="ally-portal-frontend-dev"
)

ally_config_loader = OpenAPIToolsLoader(
    openapi_url="https://ally-config-ui.dev.copilot.aws.inform-cloud.io/openapi.json",
    models_filename="ally_config_api_models.py",
    regenerate_models=True  # Generate the models file
)
print("✅ Ally Config loader initialized.")
ally_config_tools = ally_config_loader.load_tools()

print(f"✅ Loaded {len(ally_config_tools)} Ally Config tools!")


Loading Ally Config API tools...
✅ Ally Config loader initialized.
Generating models file 'ally_config_api_models.py' from https://ally-config-ui.dev.copilot.aws.inform-cloud.io/openapi.json...
Successfully generated 'ally_config_api_models.py'
Created tool: get_aws_logs_api_getAWSLogs_post [POST /api/getAWSLogs]
Created tool: get_capabilities_api_capabilities_get [GET /api/capabilities]
Created tool: get_active_config_api_getActiveConfig_post [POST /api/getActiveConfig]
Created tool: validate_config_api_validateConfig_post [POST /api/validateConfig]
Created tool: set_config_api_setConfig_post [POST /api/setConfig]
Created tool: get_config_history_api_getConfigHistory_post [POST /api/getConfigHistory]
Created tool: create_endpoint_api_createEndpoint_post [POST /api/createEndpoint]
Created tool: get_endpoint_metadata_api_getEndpointMetadata_post [POST /api/getEndpointMetadata]
Created tool: update_endpoint_metadata_api_updateEndpointMetadata_post [POST /api/updateEndpointMetadata]
Creat

In [4]:
print("🚀 Practical tool execution pattern:\n")

async def execute_tool_safely(tool, **kwargs):
    """
    Safely execute a tool with proper error handling
    """
    try:
        # Create dependencies and proper RunContext
        dependencies = ally_config_loader.create_dependencies(auth_manager=auth_manager)
        
        # Create a proper RunContext with dependencies
        class MockRunContext:
            def __init__(self, deps):
                self.deps = deps
        
        ctx = MockRunContext(dependencies)
        
        print(f"🔄 Attempting to call: {tool.name}")
        result = await tool.function(ctx, **kwargs)
        print(f"✅ Success! Result type: {type(result)}")
        return result
        
    except Exception as e:
        print(f"⚠️ Error calling {tool.name}: {str(e)}")
        return None

🚀 Practical tool execution pattern:



In [15]:
get_available_ai_models_tool = ally_config_loader.get_tool_by_operation_id("get_available_AI_models_api_getAvailableAIModels_post")
print(await execute_tool_safely(get_available_ai_models_tool))
print(get_available_ai_models_tool.tool_def)

🔄 Attempting to call: get_available_AI_models_api_getAvailableAIModels_post
Obtaining authorization token...
⠋ waiting for authorization...

'\\wsl.localhost\Ubuntu-24.04\home\jschillb\meta_ally\examples'
CMD.EXE was started with the above path as the current directory.
UNC paths are not supported.  Defaulting to Windows directory.


Token obtained, expires at: 1762862331.6426153    
✅ Success! Result type: <class 'dict'>
{'success': True, 'error_msg': None, 'data': [{'label': 'gpt-4o', 'name': 'ChatGPT 4o', 'description': 'ChatGPT 4 Omni - Use gpt-4o-latest instead!', 'enabled': False}, {'label': 'gpt-4o-latest', 'name': 'ChatGPT 4o', 'description': 'ChatGPT 4o (€€€€€)\n', 'enabled': True}, {'label': 'gpt-4o-mini', 'name': 'ChatGPT 4o mini', 'description': 'ChatGPT 4o mini (€) Retirement planned for 2025-09-15\n', 'enabled': False}, {'label': 'gpt-4.1-nano', 'name': 'GPT 4.1 nano', 'description': 'GPT 4.1 nano (€)\nThe gpt-4.1 series is the latest iteration of the gpt-4o model family. gpt-4.1-nano supports both text and vision provides at even lower cost and latency compared to the gpt-4o and gpt-4o-mini models.\nhttps://aka.ms/gpt-4-dot-1-blog\n', 'enabled': True}, {'label': 'gpt-4.1-mini', 'name': 'GPT 4.1 mini', 'description': 'GPT 4.1 mini (€€)\nThe gpt-4.1 series is the latest iteration of the gpt-4o model fa

In [11]:
create_endpoint_tool = ally_config_loader.get_tool_by_operation_id("create_endpoint_api_createEndpoint_post")
print(create_endpoint_tool.tool_def)

ToolDefinition(name='create_endpoint_api_createEndpoint_post', parameters_json_schema={'type': 'object', 'properties': {'endpoint': {'type': 'string', 'title': 'Endpoint', 'description': 'Name of the endpoint'}, 'endpoint_attributes': {'anyOf': [{'properties': {'dep_name': {'type': 'string', 'title': 'Dep Name', 'description': 'Model of the endpoint'}, 'instructions': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'title': 'Instructions', 'description': 'Instructions for the endpoint'}, 'default_message': {'type': 'string', 'title': 'Default Message', 'description': 'Default message for the endpoint'}}, 'type': 'object', 'required': ['dep_name', 'default_message'], 'title': 'EndpointAttributes', 'description': 'Dataclass for attributes needed to create an endpoint.'}, {'type': 'null'}], 'description': 'Attributes of the endpoint'}, 'endpoint_metadata': {'anyOf': [{'properties': {'display_name': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'title': 'Display Name', 'description':

In [6]:
# Load tools from AI Knowledge API
print("Loading AI Knowledge API tools...")

ai_knowledge_loader = OpenAPIToolsLoader(
    openapi_url="https://backend-api.dev.ai-knowledge.aws.inform-cloud.io/openapi.json",
    models_filename="ai_knowledge_api_models.py",
    regenerate_models=True  # Generate the models file
)
print("✅ AI Knowledge loader initialized.")
ai_knowledge_tools = ai_knowledge_loader.load_tools()

print(f"✅ Loaded {len(ai_knowledge_tools)} AI Knowledge tools!")

Loading AI Knowledge API tools...
✅ AI Knowledge loader initialized.
Generating models file 'ai_knowledge_api_models.py' from https://backend-api.dev.ai-knowledge.aws.inform-cloud.io/openapi.json...
Successfully generated 'ai_knowledge_api_models.py'
Created tool: list_collections [GET /collections/]
Created tool: create_collection [POST /collections/]
Created tool: get_collection_configuration [GET /collections/{collection_id}]
Created tool: update_collection [PUT /collections/{collection_id}]
Created tool: delete_collection [DELETE /collections/{collection_id}]
Created tool: index_collection [POST /collections/{collection_id}/index]
Created tool: query [POST /collections/{collection_id}/query]
Created tool: get_fragment_counts [GET /collections/{collection_id}/fragments/counts]
Created tool: get_collection_usage [GET /collections/{collection_id}/usage]
Created tool: list_sources [GET /sources/]
Created tool: create_source [POST /sources/]
Created tool: get_source_configuration [GET /

In [12]:
create_collection_tool = ai_knowledge_loader.get_tool_by_operation_id("create_collection")
print(create_collection_tool.tool_def)

ToolDefinition(name='create_collection', parameters_json_schema={'type': 'object', 'properties': {'collectionId': {'type': 'string', 'maxLength': 62, 'minLength': 8, 'pattern': '^[a-zA-Z_][a-zA-Z0-9_]*$', 'title': 'Collectionid', 'description': 'A unique identifier for the collection.'}, 'description': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'title': 'Description', 'description': 'Description of the collection.'}, 'projectNumber': {'type': 'string', 'pattern': '^(00000|[1-9]\\d{4}|1\\d{5})$', 'title': 'Projectnumber', 'description': 'Project number related to the collection. This number is used to derive the business department (GB) and to allocate costs.', 'default': '00000'}, 'collectionType': {'oneOf': [{'properties': {'databaseType': {'type': 'string', 'title': 'Databasetype', 'default': 'postgresql-pgvector', 'enum': ['postgresql-pgvector']}, 'vectorType': {'type': 'string', 'enum': ['vector', 'halfvec', 'bit', 'sparsevec'], 'title': 'Vectortype', 'description': "The typ

In [13]:
create_source_tool = ai_knowledge_loader.get_tool_by_operation_id("create_source")
print(create_source_tool.tool_def)

ToolDefinition(name='create_source', parameters_json_schema={'type': 'object', 'properties': {'sourceId': {'type': 'string', 'maxLength': 255, 'minLength': 8, 'pattern': '^[a-zA-Z0-9-_]+$', 'title': 'Sourceid', 'description': 'A unique identifier for the data source.'}, 'description': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'title': 'Description', 'description': 'Description of the data source.'}, 'properties': {'oneOf': [{'properties': {'prefix': {'items': {'type': 'string'}, 'type': 'array', 'maxItems': 32, 'minItems': 0, 'title': 'Prefix', 'description': 'A list of prefixes to filter objects in the source.', 'default': []}, 'suffix': {'items': {'type': 'string'}, 'type': 'array', 'maxItems': 32, 'minItems': 0, 'title': 'Suffix', 'description': 'A list of suffixes to filter objects in the source.', 'default': []}, 'prefixIgnore': {'items': {'type': 'string'}, 'type': 'array', 'maxItems': 32, 'minItems': 0, 'title': 'Prefixignore', 'description': 'A list of prefixes to filte

In [14]:
update_collection_tool = ai_knowledge_loader.get_tool_by_operation_id("update_collection")
print(update_collection_tool.tool_def.parameters_json_schema)

{'type': 'object', 'properties': {'collection_id': {'type': 'string', 'description': ''}, 'collectionId': {'type': 'string', 'maxLength': 62, 'minLength': 8, 'pattern': '^[a-zA-Z_][a-zA-Z0-9_]*$', 'title': 'Collectionid', 'description': 'A unique identifier for the collection.'}, 'description': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'title': 'Description', 'description': 'Description of the collection.'}, 'projectNumber': {'type': 'string', 'pattern': '^(00000|[1-9]\\d{4}|1\\d{5})$', 'title': 'Projectnumber', 'description': 'Project number related to the collection. This number is used to derive the business department (GB) and to allocate costs.', 'default': '00000'}, 'collectionType': {'oneOf': [{'properties': {'databaseType': {'type': 'string', 'title': 'Databasetype', 'default': 'postgresql-pgvector', 'enum': ['postgresql-pgvector']}, 'vectorType': {'type': 'string', 'enum': ['vector', 'halfvec', 'bit', 'sparsevec'], 'title': 'Vectortype', 'description': "The type of vec

In [7]:
async def execute_ai_knowledge_tool_safely(tool, **kwargs):
    """
    Safely execute an AI Knowledge tool with proper error handling
    """
    try:
        # Create dependencies and proper RunContext for AI Knowledge
        dependencies = ai_knowledge_loader.create_dependencies(auth_manager=auth_manager)
        
        # Create a proper RunContext with dependencies
        class MockRunContext:
            def __init__(self, deps):
                self.deps = deps
        
        ctx = MockRunContext(dependencies)
        
        print(f"🔄 Attempting to call: {tool.name}")
        result = await tool.function(ctx, **kwargs)
        print(f"✅ Success! Result type: {type(result)}")
        return result
        
    except Exception as e:
        print(f"⚠️ Error calling {tool.name}: {str(e)}")
        return None

# Execute AI Knowledge tool
list_sources_tool = ai_knowledge_loader.get_tool_by_operation_id("list_sources")
await execute_ai_knowledge_tool_safely(list_sources_tool)

🔄 Attempting to call: list_sources
✅ Success! Result type: <class 'dict'>


{'Count': 34,
 'ScannedCount': 62,
 'TotalCount': 34,
 'Items': [{'sourceId': 'Addone_Github',
   'sourceType': 'github',
   'description': 'For testing purpose',
   'public': False,
   'accessDenied': True},
  {'sourceId': 'angular_docs',
   'sourceType': 'website',
   'description': 'Official Angular Documentation',
   'public': False,
   'accessDenied': True},
  {'sourceId': 'Benchmark',
   'sourceType': 'sharepoint',
   'description': None,
   'public': False,
   'accessDenied': True},
  {'sourceId': 'Benchmark_no_htm',
   'sourceType': 'sharepoint',
   'description': None,
   'public': False,
   'accessDenied': True},
  {'sourceId': 'e2e_test_source_108',
   'sourceType': 'website',
   'description': 'E2E Test Source for Playwright, created 11/5/2025, 1:14:19 PM. Feel free to delete.',
   'public': True,
   'accessDenied': False},
  {'sourceId': 'e2e_test_source_6729',
   'sourceType': 'website',
   'description': 'E2E Test Source for Playwright, created 11/5/2025, 1:12:15 PM. Fee

# Using the ToolGroupManager

Now let's demonstrate how to use the ToolGroupManager which provides a more organized way to work with API tools and includes convenient methods for executing tools by operation ID.

In [8]:
# Import and initialize the ToolGroupManager
from src.util.tool_group_manager import ToolGroupManager

print("Setting up ToolGroupManager...")
tool_manager = ToolGroupManager(auth_manager=auth_manager)

# Load both API tools
print("Loading AI Knowledge tools...")
tool_manager.load_ai_knowledge_tools(regenerate_models=False)  # Use existing models

print("Loading Ally Config tools...")
tool_manager.load_ally_config_tools(regenerate_models=False)  # Use existing models

print("✅ ToolGroupManager setup complete!")

Setting up ToolGroupManager...
Loading AI Knowledge tools...
Created tool: list_collections [GET /collections/]
Created tool: create_collection [POST /collections/]
Created tool: get_collection_configuration [GET /collections/{collection_id}]
Created tool: update_collection [PUT /collections/{collection_id}]
Created tool: delete_collection [DELETE /collections/{collection_id}]
Created tool: index_collection [POST /collections/{collection_id}/index]
Created tool: query [POST /collections/{collection_id}/query]
Created tool: get_fragment_counts [GET /collections/{collection_id}/fragments/counts]
Created tool: get_collection_usage [GET /collections/{collection_id}/usage]
Created tool: list_sources [GET /sources/]
Created tool: create_source [POST /sources/]
Created tool: get_source_configuration [GET /sources/{source_id}]
Created tool: update_source [PUT /sources/{source_id}]
Created tool: delete_source [DELETE /sources/{source_id}]
Created tool: health_check [GET /api/test]
Created tool:

In [9]:
# Example 1: Get a tool by operation ID
print("🔍 Finding tool by operation ID...")
tool = tool_manager.get_tool_by_operation_id("get_available_AI_models_api_getAvailableAIModels_post")
if tool:
    print(f"✅ Found tool: {tool.name}")
else:
    print("❌ Tool not found")

# Example 2: Execute a tool safely using the convenience method
print("\n🚀 Executing tool using execute_tool_safely method...")
result = await tool_manager.execute_tool_safely("get_available_AI_models_api_getAvailableAIModels_post")
if result:
    print(f"📊 Tool execution successful! Result has {len(result) if isinstance(result, list) else 'data'}")
else:
    print("❌ Tool execution failed or returned None")

🔍 Finding tool by operation ID...
✅ Found tool: get_available_AI_models_api_getAvailableAIModels_post

🚀 Executing tool using execute_tool_safely method...
🔄 Attempting to call: get_available_AI_models_api_getAvailableAIModels_post
✅ Success! Result type: <class 'dict'>
📊 Tool execution successful! Result has data


In [10]:
# Example 3: Execute an AI Knowledge tool
print("🧠 Trying AI Knowledge tool...")
ai_result = await tool_manager.execute_tool_safely("list_sources")
if ai_result:
    print(f"✅ AI Knowledge tool successful!")
else:
    print("ℹ️ AI Knowledge tool may require additional parameters or authentication")

# Example 4: Show available tool groups information
print("\n📋 Available tool groups:")
groups_info = tool_manager.get_available_groups()

print(f"AI Knowledge groups: {list(groups_info['ai_knowledge_groups'].keys())}")
print(f"Ally Config groups: {list(groups_info['ally_config_groups'].keys())}")

# Show specific tools in a group (example: sources)
if 'sources' in groups_info['ai_knowledge_groups']:
    sources_tools = groups_info['ai_knowledge_groups']['sources']
    print(f"\nSource management tools: {sources_tools}")

🧠 Trying AI Knowledge tool...
🔄 Attempting to call: list_sources
✅ Success! Result type: <class 'dict'>
✅ AI Knowledge tool successful!

📋 Available tool groups:
AI Knowledge groups: ['sources', 'documents', 'search', 'retrieval', 'processing', 'metadata', 'collections', 'indexing', 'permissions', 'admin']
Ally Config groups: ['endpoints', 'configs', 'evaluations', 'permissions', 'audit', 'costs', 'inventory']

Source management tools: ['list_sources', 'create_source', 'get_source_configuration', 'update_source', 'delete_source']
